In [7]:
#Importing so maybe VS code will let me debug the code
import ipykernel
ipykernel.__version__

'6.9.1'

In [8]:
import csv
import sys
import pandas as pd
import numpy as np

# Importing the files in different formats

In [9]:
from file_dir import video_dir
points = ['world_index', 'gaze_normal0_x', 'gaze_normal0_y', 'gaze_normal0_z']

# The dataframe I plan on using
# gaze_normal = pd.read_csv(video_dir, usecols=points)

#A simplified table that consists of the x and y positions in the world image in normalized coordinates 
#norm_pt_x = ['world_index', 'norm_pos_x']
#norm_pt_y = ['world_index', 'norm_pos_y']
#norm_pt_w = ['world_index']
#norm_pt = ['world_index', 'norm_pos_x', 'norm_pos_y']

gaze_pos = pd.read_csv('C:/Users/deadg/OneDrive/Documents/GithubRep/SeniorCapstone/TextExtraction/ExportedData/OldTestData/gaze_positions.csv', usecols=points)



Narrowing down data selection

In [10]:
#To complete step 2 we have to figure out the data looks like when the user is focused on an object
#The distance or size of an object is goind to present an issue with the way I have decided to select the frame
pd.set_option("display.max_rows", None, "display.max_columns", None)
gaze_pos.head()
pos_x = gaze_pos["gaze_normal0_x"]
pos_y = gaze_pos["gaze_normal0_y"]
pos_z = gaze_pos["gaze_normal0_z"]
w_index = gaze_pos["world_index"]
gaze_pos.head()

,world_index,gaze_normal0_x,gaze_normal0_y,gaze_normal0_z
0,0,-0.016253,-0.170459,0.985231
1,0,-0.021103,-0.174047,0.984511
2,1,-0.013837,-0.174720,0.984521
3,1,-0.013466,-0.174308,0.984599
4,2,-0.013363,-0.171942,0.985016


Getting the index of the all the True values from tOrF and adding them to a list

In [11]:
#IT WORKS THIS IS IT BABY
# This takes out all the false values from the tOrF series
def frame_Extract(tOrF):
    list_Index = []
    count = 0 
    # Based on the size of the initial data, It runs through the list and gets the indexed number of each value that is equal to True and adds it to the list. 
    while count != tOrF.size:
        if tOrF.get(count) == True:
            #print("yes", y)
            list_Index.append(count)
        count += 1 #counter for the while loop
    
    # Splits the big list into separate list and adds them into a dictionary to be sorted
    # Still throws a index out of bounds execption tho in the
    gFrame = {}
    tempGroup = []
    count = 0
    groupNum = 0
    while count != len(list_Index) - 1:
        #if the next number is the expected number then the current number gets added to the list.
        if list_Index[count + 1] == list_Index[count] + 1:
            tempGroup.append(list_Index[count])
        else:
            # adds the last value to the current group before moving onto the next group of values.
            tempGroup.append(list_Index[count])
            gFrame["group{0}".format(groupNum)] = tempGroup
            tempGroup = []
            groupNum += 1
        count += 1

    # Time to clean up that data! Getting rid of the groups that have fewer then 10 items
    # Source: https://www.geeksforgeeks.org/python-ways-to-remove-a-key-from-dictionary/
    almostDone = {key: val for key, val in gFrame.items() if len(val) > 10}
    return match_Up(mid_part1(almostDone))

Finding the middle of each group of numbers

In [12]:
# Takes the cleaned Data and goes through each key value pair and sends the values to findMiddle
def mid_part1(almostDone):
    m_list = []
    for key in sorted(almostDone):
        m_list.append(mid_part2(almostDone[key]))
    return m_list

In [13]:
# Finds the middle element in the list and returns it.
# https://stackoverflow.com/questions/38130895/find-middle-of-a-list
def mid_part2(input_list):
    middle = float(len(input_list))/2
    if middle % 2 != 0:
        return input_list[int(middle - .5)]
    else:
        return input_list[int(middle)]

Getting the World_index for the Frames

In [14]:
# matching the selected coordinate indexes to its corresponding world_index 
def match_Up(mid_index):
    mid_Frame = []
    for x in mid_index:
        mid_Frame.append(w_index.get(x))
    return mid_Frame

In [15]:
def set_X_coords(lowest, highest):
    tOrF_X = pos_x.between(lowest, highest)
    return tOrF_X

In [16]:
def set_Y_coords(lowest, highest):
    tOrF_Y = pos_y.between(lowest, highest)
    return tOrF_Y

In [17]:
def set_Z_coords(lowest, highest):
    tOrF_Z = pos_z.between(lowest, highest)
    return tOrF_Z

# Main
Could do the same thing for the accuracy value too.
This program will take the X and Y coordinates from gaze_positions.csv and find the middle most frame of each grouped frame.

In [20]:
# Using the between method to make a true or false list based on a given range
#These are the values that we will need to change.
highest = .50
lowest = .40
# For the X coordinates
tOrF_X = set_X_coords(lowest, highest)# is a list of true or false. if the value in pos_x fits within that range it becomes true if not its false.
# For the Y coordinates
tOrF_Y = set_Y_coords(lowest, highest)

tOrF_Z = set_Z_coords(lowest, highest)

# Just double checking lengths
# Combined_TF is a combinded list of the tOrF list for the x, y and z coordinates.
if len(tOrF_X) == len(tOrF_Y):
    Combined_TF = tOrF_X.eq(tOrF_Y)
    Combined_TF.eq(tOrF_Z)
else:
    print("YOU DONT WANT TO SEE THIS!!!!!!!!!!!!!!!!!!!!!!!!")
# Sending the data through the methods

mid_combined = frame_Extract(Combined_TF)
print("The Combined list of coordinates is ", mid_combined)

# Not sure if we still need this but it doesn't hurt
''' For some reason idk why this is having an issue 
mid_x = frame_Extract(tOrF_X)
print("The X coordinate Frames are ", mid_x)
mid_y = frame_Extract(tOrF_Y)
print("The Y coordinate Frames are ", mid_y)
mid_z = frame_Extract(tOrF_Z)
print("The Z coordinate Frames are ", mid_z)
'''


0        True
1        True
2        True
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13       True
14       True
15       True
16       True
17       True
18       True
19       True
20       True
21       True
22       True
23       True
24       True
25       True
26       True
27       True
28       True
29       True
30       True
31       True
32       True
33       True
34       True
35       True
36       True
37       True
38       True
39       True
40       True
41       True
42       True
43       True
44       True
45       True
46       True
47       True
48       True
49       True
50       True
51       True
52       True
53       True
54       True
55       True
56       True
57       True
58       True
59       True
60       True
61       True
62       True
63       True
64       True
65       True
66       True
67       True
68       True
69       True
70       True
71    

' For some reason idk why this is having an issue \nmid_x = frame_Extract(tOrF_X)\nprint("The X coordinate Frames are ", mid_x)\nmid_y = frame_Extract(tOrF_Y)\nprint("The Y coordinate Frames are ", mid_y)\nmid_z = frame_Extract(tOrF_Z)\nprint("The Z coordinate Frames are ", mid_z)\n'